In [12]:
import torch
import pyaudio
import wave
import os
import time
from AudioUtil import AudioUtil
from AudioClassifier import AudioClassifier

print(torch.__version__)
model = AudioClassifier()
state_dict = torch.load("data/models/V8_model_state_dict.pth")
model.load_state_dict(state_dict)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
model.eval()

CHUNK = 1050
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100

2.3.0+cu118
Using device: cuda:0


In [13]:
def record_audio(duration=2):
    frames = []
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    try:
        # Calculate the correct number of iterations to cover the duration
        num_frames = int((RATE / CHUNK) * duration)
        for _ in range(num_frames):
            data = stream.read(CHUNK)
            frames.append(data)
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()
    return frames

def save_temp_audio(frames, filename="temp_audio.wav"):
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(pyaudio.PyAudio().get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

def process_audio_file(filename):
    class_names = {0: 'Noise', 1: 'Knock'} 
    aud = AudioUtil.open(filename)
    # os.remove(filename)
    reaud = AudioUtil.resample(aud, RATE)
    rechan = AudioUtil.rechannel(reaud, CHANNELS)
    dur_aud = AudioUtil.pad_trunc(rechan, 2000)
    sgram = AudioUtil.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram = sgram.to(device)

    outputs = model(sgram.unsqueeze(0))

    # Get the predicted class with the highest score
    _, prediction = torch.max(outputs, 1)
    predicted_classes = [class_names.get(p.item(), p.item()) for p in prediction]
    # print(prediction.item())
    print(predicted_classes)
    # if prediction > 0.5:
    #     print("Klopfen erkannt!")

In [17]:
# process_audio_file for every file in train
for file in os.listdir("data/validation"):
    print(f"Processing {file}")
    process_audio_file(f"data/validation/{file}")
# process_audio_file("data/knocks/knock_10.wav")
# process_audio_file("data/noises/noise_72.wav")

Processing knocks_with_noise_0_var5.wav
['Knock']
Processing knocks_with_noise_10_var7.wav
['Knock']
Processing knocks_with_noise_10_var8.wav
['Knock']
Processing knocks_with_noise_11_var1.wav
['Knock']
Processing knocks_with_noise_12_var0.wav
['Knock']
Processing knocks_with_noise_12_var3.wav
['Knock']
Processing knocks_with_noise_12_var7.wav
['Knock']
Processing knocks_with_noise_13.wav
['Knock']
Processing knocks_with_noise_13_var0.wav
['Knock']
Processing knocks_with_noise_13_var8.wav
['Knock']
Processing knocks_with_noise_15_var7.wav
['Knock']
Processing knocks_with_noise_16_var3.wav
['Knock']
Processing knocks_with_noise_17_var3.wav
['Knock']
Processing knocks_with_noise_17_var5.wav
['Knock']
Processing knocks_with_noise_17_var6.wav
['Knock']
Processing knocks_with_noise_17_var9.wav
['Knock']
Processing knocks_with_noise_18_var1.wav
['Knock']
Processing knocks_with_noise_18_var2.wav
['Knock']
Processing knocks_with_noise_18_var3.wav
['Knock']
Processing knocks_with_noise_18_var6.

In [5]:
print("Starte Klopfen-Erkennung...")
while True:
    start_time = time.time()
    # frames = record_audio()
    # save_temp_audio(frames)
    # process_audio_file("temp_audio.wav")
    # process_audio_file("noise_1.wav")
    process_audio_file("knock_1.wav")
    print(f"Time taken: {time.time() - start_time}")

Starte Klopfen-Erkennung...
[4]
Time taken: 0.01252126693725586
[4]
Time taken: 0.011546134948730469
[4]
Time taken: 0.011914968490600586
[4]
Time taken: 0.005639791488647461
[4]
Time taken: 0.005643606185913086
[4]
Time taken: 0.006499528884887695
[4]
Time taken: 0.0055086612701416016
[4]
Time taken: 0.007029056549072266
[4]
Time taken: 0.00598597526550293
[4]
Time taken: 0.006001472473144531
[4]
Time taken: 0.0055086612701416016
[4]
Time taken: 0.006011486053466797
[4]
Time taken: 0.006056785583496094
[4]
Time taken: 0.005463838577270508
[4]
Time taken: 0.00604248046875
[4]
Time taken: 0.006995439529418945
[4]
Time taken: 0.0050013065338134766
[4]
Time taken: 0.006001949310302734
[4]
Time taken: 0.006014585494995117
[4]
Time taken: 0.006111621856689453
[4]
Time taken: 0.01150059700012207
[4]
Time taken: 0.005576372146606445
[4]
Time taken: 0.005681037902832031
[4]
Time taken: 0.005502939224243164
[4]
Time taken: 0.0065076351165771484
[4]
Time taken: 0.005540370941162109
[4]
Time take

KeyboardInterrupt: 